In [4]:
import os,tables,numpy as np,matplotlib.pyplot as plt,pandas as pd
from scipy.io import loadmat
from scipy.signal import resample
import scipy.io.wavfile as wav
from collections import Counter
import h5py,glob,json
import sounddevice as sd
from matplotlib import cm
from math import ceil
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import math

In [2]:
from __future__ import print_function, absolute_import, division
from keras import backend as K
from keras.engine.base_layer import Layer
from keras.engine.topology import InputSpec
import tensorflow as tf
from keras.layers.merge import Concatenate
from keras.utils import conv_utils
from keras.layers import Input, MaxPooling1D ,Conv1D,Activation,Dense, activations, initializers, Flatten, regularizers, constraints, Lambda,Dropout
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
from keras.models import Model
from keras.utils import plot_model
import numpy as np
from scipy.fftpack import dct
from keras.backend.common import normalize_data_format
from keras.layers.merge import Multiply
from keras.utils import to_categorical
from keras.callbacks import LearningRateScheduler

In [3]:
class MFCC(Layer):
    '''The number of output shape must be fixed. can't be learnable. 
    So the number of coefficients and frames should be fixed.

    
    Linear learnable weight vector , does dot multiplication on a vector
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).'''

    def __init__(self, weights=None, 
                 axis=-1,init='he_normal',
                 samplerate=1000,winlen=0.025,
                 winstep=0.01,numcep=13,nfilt=26
                 nfft=512,lowfreq=0,highfreq=None,preemph=0.97,
                 ceplifter=22,winfunc=lambda x:numpy.ones((x,)),
                 appendEnergy = True,
                 **kwargs):
        self.axis = axis
        self.init = initializers.get(init)
        self.kernel = weights
        self.samplerate = samplerate
        self.winlen = winlen
        self.winstep = winstep
        super(MFCC, self).__init__(**kwargs)
    def build(self, input_shape):
        if len(input_shape)>3:
            raise ValueError("Input should have 2 axes")
        elif len(input_shape)<2:
            raise ValueError("Input layer is flattened, should have 2 axes")
        self.input_dim = input_shape[-1]            
        self.kernel = self.add_weight(shape=(self.input_dim,),
                                      initializer=initializers.Ones(),
                                      name='kernel',
                                      constraint=constraints.NonNeg()
                                      #constraint=constraints.min_max_norm(min_value=0.0, max_value=1.0)
                                      #constraint=constraints.UnitNorm(axis=self.axis)
                                     )
        self.input_spec = InputSpec(min_ndim=2, axes={-1: self.input_dim})            
        self.built = True
    def call(self, inputs):
        output = tf.multiply(inputs,self.kernel)
        return output
    def compute_output_shape(self, input_shape):
        return input_shape
    def get_config(self):
        config = {"kernel": self.kernel, "axis": self.axis}
        base_config = super(Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [5]:
folder_idx = 'a'
datapath='/media/mhealthra2/Data/heart_sound/Heart_Sound/Physionet/training/training-'+folder_idx+'/';
labelpath="/media/mhealthra2/Data/heart_sound/Heart_Sound"\
           "/Physionet/2016-07-25_Updated files for Challenge 2016"\
           "/20160725_Reference with signal quality results for training set"\
           "/training-"+folder_idx+ "/REFERENCE_withSQI.csv"
Fs = 1000
files = [x for x in os.listdir(datapath)if x.endswith('wav')]
labels = pd.read_csv(labelpath,names=['wav','class','quality'])
(fs,sig) = wav.read(os.path.join(datapath,files[0]))
x = sig[:2500]

In [6]:
xx = tf.convert_to_tensor(x.astype(np.float32))

In [25]:
winlen = 0.025
winstep = 0.01
numcep = 13
samplerate = 1000
nfilt = 26
nfft = 512
lowfreq = 0
highfreq = 500
ceplifter = 22
winfunc = lambda x:numpy.ones((x,))

In [26]:
import decimal
def round_half_up(number):
    return int(decimal.Decimal(number).quantize(decimal.Decimal('1'), rounding=decimal.ROUND_HALF_UP))

In [28]:
slen = xx.get_shape()[0].value
frame_len = winlen*samplerate
frame_step = winstep*samplerate
frame_len = int(round_half_up(frame_len))
frame_step = int(round_half_up(frame_step))

In [18]:
a.value

2500